In [ ]:
%pylab inline
import time
import numpy as np
import matplotlib.pyplot as plt
t_deb = time.time()

T=0
T=10.0 
h=0.03
Nt=100 
N_mc=100 
EPS=0.0001 
N0=30
N1=30
N2=45

H=6
H0=3/30
h0=H/N0 
h1=H/N1 
h2=H0/N2

eps=H0/5 
nu=0.01
U0=3.0
G0=1.0
sig=0.01

sqrt2nu=np.sqrt(2*nu)
sqrth=np.sqrt(h)
inv2pi = 1/(2*np.pi)

def THETA_0(x): #initial boundary vorticity
    return -U0*(3*np.ones_like(x)-x/H)

Xbu=np.zeros((2*N1+1,N2+1,2)) #X for boundary layer
Xbd=np.zeros((2*N1+1,N2+1,2)) 
Xou=np.zeros((2*N0+1,N0+1,2)) #X for outer layer
Xod=np.zeros((2*N0+1,N0+1,2)) 
Wb=np.zeros((2*N1+1,N2+1)) #Initial vorticity in boundary layer
Wo=np.zeros((2*N0+1,N0+1)) #Initial vorticity in outer layer
Gb=np.zeros((2*N1+1,N2+1)) #External force in boundary layer
Go=np.zeros((2*N0+1,N0+1)) #External force in outer layer

INDbu=np.ones((2*N1+1,N2+1)) #indicators for Xb
INDbu[:,0]=np.zeros(2*N1+1)
INDbd=np.ones((2*N1+1,N2+1)) #indicators for Xb
INDbd[:,0]=np.zeros(2*N1+1)
INDou=np.ones((2*N0+1,N0+1)) #indicators for Xo
INDou[:,0]=np.zeros(2*N0+1)
INDod=np.ones((2*N0+1,N0+1)) #indicators for Xo
INDod[:,0]=np.zeros(2*N0+1)

for j1 in range(2*N1+1): #initialise Xb
    for j2 in range(N2+1): 
        Xbu[j1,j2,:]=[(j1-N1)*h1, j2*h2]
        Xbd[j1,j2,:]=[(j1-N1)*h1, -j2*h2]
for j1 in range(2*N0+1): #initialise Xo
    for j2 in range(N0+1):
        Xou[j1,j2,:]=[(j1-N0)*h0, j2*h0]
        Xod[j1,j2,:]=[(j1-N0)*h0, -j2*h0]

phi=np.zeros((2*N1+1,N2+1))
for j1 in range(2*N1+1):
    for j2 in range(N2+1):
        if(Xbu[j1,j2,1]>eps/3 and Xbu[j1,j2,1]<2*eps/3):
            phi[j1,j2]=THETA_0(Xbu[j1,j2,0])*nu/(eps**2)*324*(Xbu[j1,j2,1]/eps-0.5)
phi=np.reshape(phi,((2*N1+1)*(N2+1)))

x3_0=np.copy(Xbu[:,0,:])
Xbu=np.reshape(Xbu,((2*N1+1)*(N2+1),2))
Xbd=np.reshape(Xbd,((2*N1+1)*(N2+1),2))
Xou=np.reshape(Xou,((2*N0+1)*(N0+1),2))
Xod=np.reshape(Xod,((2*N0+1)*(N0+1),2))
Gb=np.reshape(Gb,((2*N1+1)*(N2+1)))
Go=np.reshape(Go,((2*N0+1)*(N0+1)))
psi=np.copy(Go)
psi_b=np.copy(Gb)

for j1 in range(2*N1+1): #initialise Wb
    for j2 in range(N2+1):
        Wb[j1,j2]=-U0*(3.0-(j1-N1)*h1/H)*(1.0-j2*h2)*(j2*h2<=1.0)
for j1 in range(2*N0+1): #initialise Wo 
    for j2 in range(N0+1):
        Wo[j1,j2]=-U0*(3.0-(j1-N0)*h0/H)*(1.0-j2*h0)*(j2*h0<=1.0)

Wb=np.reshape(Wb,(2*N1+1)*(N2+1))
Wo=np.reshape(Wo,(2*N0+1)*(N0+1))
INDbu=np.reshape(INDbu,(2*N1+1)*(N2+1))
INDbd=np.reshape(INDbd,(2*N1+1)*(N2+1))
INDou=np.reshape(INDou,(2*N0+1)*(N0+1))
INDod=np.reshape(INDod,(2*N0+1)*(N0+1))
            
Ub=np.zeros((2*N1+2,N2+2,2)) 
Uo=np.zeros((2*N0+2,N0+2,2)) 
for j1 in range(2*N0+2): 
    for j2 in range(N0+2):
        Uo[j1,j2]=[(j1-N0)*h0, j2*h0]
for j1 in range(2*N1+2): 
    for j2 in range(N2+2): 
        Ub[j1,j2]=[(j1-N1)*h1, j2*h2*5]
Uo=np.reshape(Uo,((2*N0+2)*(N0+2),2))
Ub=np.reshape(Ub,((2*N1+2)*(N2+2),2))

X_th=np.zeros((10*N1+1,2)) 
for j1 in range(10*N1+1): 
    X_th[j1,:]=[(j1-5*N1)*h1/5, 0]
    
THETA=np.zeros((Nt-1,10*N1))

x1=np.array(Uo[:,0])
y1=np.array(Uo[:,1])
x2=np.array(Ub[:,0])
y2=np.array(Ub[:,1])

def K(x,y): #Biot-Savart kernel
    X1 = np.tensordot(x[:,0], np.ones_like(y[:,0]), axes=0)
    X2 = np.tensordot(x[:,1], np.ones_like(y[:,1]), axes=0)
    Y1 = np.tensordot(np.ones_like(x[:,0]), y[:,0], axes=0)
    Y2 = np.tensordot(np.ones_like(x[:,1]), y[:,1], axes=0)

    Sp = (X1-Y1)**2+(X2+Y2)**2
    Sn = (X1-Y1)**2+(X2-Y2)**2
    
    Moll_p = (Sp!=0).astype(float)*(1.0-np.exp(-Sp**2/sig))/(Sp+(Sp==0).astype(float))
    Moll_n = (Sn!=0).astype(float)*(1.0-np.exp(-Sn**2/sig))/(Sn+(Sn==0).astype(float))

    K = np.zeros((len(x[:,0]), len(y[:,0]), 2))
    
    K[:,:,0] = inv2pi*((Y2-X2)*Moll_n-(Y2+X2)*Moll_p)
    K[:,:,1] = inv2pi*(-(Y1-X1)*Moll_n+(Y1-X1)*Moll_p)
    return K

def u(x,Xou_temp,Xod_temp,Xbu_temp,Xbd_temp,phi_temp,psi_temp,psi_b_temp): #velocity approximation
    S=np.zeros_like(x)
    
    Iou=np.tensordot(np.ones_like(x[:,0]), (Xou_temp[:,1]>0).astype(int), axes=0) 
    Iod=np.tensordot(np.ones_like(x[:,0]), (Xod_temp[:,1]>0).astype(int), axes=0) 
    Ibu=np.tensordot(np.ones_like(x[:,0]), (Xbu_temp[:,1]>0).astype(int), axes=0) 
    Ibd=np.tensordot(np.ones_like(x[:,0]), (Xbd_temp[:,1]>0).astype(int), axes=0)
    
    Ix=2*((x[:,1]>0).astype(int)-0.5*np.ones_like(x[:,1]))
    x[:,1]=x[:,1]*Ix

    Kou=K(x,Xou_temp)
    Kod=K(x,Xod_temp)
    Kbu=K(x,Xbu_temp)
    Kbd=K(x,Xbd_temp)
    u1=np.tensordot(Iou*Kou[:,:,0]-Iod*Kod[:,:,0],Wo,axes=([1],[0]))*h0*h0+np.tensordot(Ibu*Kbu[:,:,0]-Ibd*Kbd[:,:,0],Wb,axes=([1],[0]))*h1*h2+np.tensordot(Kbu[:,:,0],psi_b_temp,axes=([1],[0]))*h1*h2*h+np.tensordot(Kou[:,:,0],psi_temp,axes=([1],[0]))*h0*h0*h+np.tensordot(Kbu[:,:,0],phi_temp,axes=([1],[0]))*h*h1*h2
    u2=np.tensordot(Iou*Kou[:,:,1]-Iod*Kod[:,:,1],Wo,axes=([1],[0]))*h0*h0+np.tensordot(Ibu*Kbu[:,:,1]-Ibd*Kbd[:,:,1],Wb,axes=([1],[0]))*h1*h2+np.tensordot(Kbu[:,:,1],psi_b_temp,axes=([1],[0]))*h1*h2*h+np.tensordot(Kou[:,:,1],psi_temp,axes=([1],[0]))*h0*h0*h+np.tensordot(Kbu[:,:,1],phi_temp,axes=([1],[0]))*h*h1*h2
    
    S[:,0]=u1
    S[:,1]=u2
    S[:,1]=S[:,1]*Ix
    x[:,1]=x[:,1]*Ix
    return S

for t in range(Nt-1):
    print(t)
    Xbu_temp=np.copy(Xbu[:,:])
    Xbd_temp=np.copy(Xbd[:,:])
    Xou_temp=np.copy(Xou[:,:])
    Xod_temp=np.copy(Xod[:,:])
    phi_temp=np.copy(phi)
    psi_temp=np.copy(psi)
    psi_b_temp=np.copy(psi_b)
    
    Xbu_upd=h*u(Xbu_temp,Xou_temp,Xod_temp,Xbu_temp,Xbd_temp,phi_temp,psi_temp,psi_b_temp)+sqrt2nu*sqrth*np.random.randn((2*N1+1)*(N2+1),2)
    Xbd_upd=h*u(Xbd_temp,Xou_temp,Xod_temp,Xbu_temp,Xbd_temp,phi_temp,psi_temp,psi_b_temp)+sqrt2nu*sqrth*np.random.randn((2*N1+1)*(N2+1),2)
    Xou_upd=h*u(Xou_temp,Xou_temp,Xod_temp,Xbu_temp,Xbd_temp,phi_temp,psi_temp,psi_b_temp)+sqrt2nu*sqrth*np.random.randn((2*N0+1)*(N0+1),2)
    Xod_upd=h*u(Xod_temp,Xou_temp,Xod_temp,Xbu_temp,Xbd_temp,phi_temp,psi_temp,psi_b_temp)+sqrt2nu*sqrth*np.random.randn((2*N0+1)*(N0+1),2)
    U1=u(Uo,Xou_temp,Xod_temp,Xbu_temp,Xbd_temp,phi_temp,psi_temp,psi_b_temp)
    U2=u(Ub,Xou_temp,Xod_temp,Xbu_temp,Xbd_temp,phi_temp,psi_temp,psi_b_temp)
    
    for j in range(len(Xbu_temp[:,0])):
        if(Xbu_temp[j,1]*(Xbu_temp[j,1]+Xbu_upd[j,1])<0):
            INDbu[j]=0.0
            phi[j]=0.0
            psi_b[j]=0.0
        if(Xbd_temp[j,1]*(Xbd_temp[j,1]+Xbd_upd[j,1])<0):
            INDbd[j]=0.0
    for j in range(len(Xou_temp[:,0])):
        if(Xou_temp[j,1]*(Xou_temp[j,1]+Xou_upd[j,1])<0):
            INDou[j]=0.0
            psi[j]=0.0
        if(Xod_temp[j,1]*(Xod_temp[j,1]+Xod_upd[j,1])<0):
            INDod[j]=0.0
    
    Xbu=Xbu_temp+Xbu_upd
    Xbd=Xbd_temp+Xbd_upd
    Xou=Xou_temp+Xou_upd
    Xod=Xod_temp+Xod_upd
    
    a1=np.array(U1[:,0]).reshape((2*N0+2,N0+2))
    b1=np.array(U1[:,1]).reshape((2*N0+2,N0+2))
    a2=np.array(U2[:,0]).reshape((2*N1+2,N2+2))
    b2=np.array(U2[:,1]).reshape((2*N1+2,N2+2))

    Wo_temp=(b1[1:,:-1]-b1[:-1,:-1])/h0-(a1[:-1,1:]-a1[:-1,:-1])/h0
    Wb_temp=(b2[1:,:-1]-b2[:-1,:-1])/h1-(a2[:-1,1:]-a2[:-1,:-1])/h2

    R=np.sqrt(4*nu*(t+1)*h)*np.random.randn((N_mc))
    THa=THETA_0(np.tensordot(Xbu[:,0], np.ones_like(R), axes=0)+np.tensordot(np.ones_like(Xbu[:,0]), R, axes=0))
    B_stress_plus=np.sum(THa,axis=1)/N_mc
    
    for j in range(len(Xbu_temp[:,0])):
        if(Xbu[j,1]>eps/3 and Xbu[j,1]<2*eps/3):
            phi[j]=phi[j]+nu/(eps**2)*B_stress_plus[j]*324*(Xbu[j,1]/eps-0.5)

    psi=psi_temp+Go
    psi_b=psi_b_temp+Gb

    M=np.average(abs(Wo_temp))
    st=M/50
    l=np.arange(-M,M,step=st)
    figure()
    plt.contourf(x1.reshape((2*N0+2,N0+2))[:-1,:-1], y1.reshape((2*N0+2,N0+2))[:-1,:-1], Wo_temp, vmax=M, vmin=-M, extend="both", cmap="coolwarm", levels=l)
    colorbar()
    st1=plt.streamplot(x1.reshape((2*N0+2,N0+2))[:-1,:-1].T, y1.reshape((2*N0+2,N0+2))[:-1,:-1].T, a1[:-1,:-1].T, b1[:-1,:-1].T,color=np.sqrt(a1[:-1,:-1].T**2+b1[:-1,:-1].T**2),cmap ='viridis',arrowsize=0.45,density=1.5)
    colorbar(st1.lines)
    plt.plot([-H,H], [0,0], linewidth=2, color="k")
    S="fig_4"+str(t)+".png"
    plt.savefig(S, dpi=300)
    plt.close

    M=np.average(abs(Wb_temp))
    st=M/50
    l=np.arange(-M,M,step=st)
    figure(figsize=(12,4))
    plt.contourf(x2.reshape((2*N1+2,N2+2))[:-1,:-1], y2.reshape((2*N1+2,N2+2))[:-1,:-1],Wb_temp,vmax=M, vmin=-M, extend="both", cmap="coolwarm", levels=l)
    colorbar()
    st3=plt.streamplot(x2.reshape((2*N1+2,N2+2))[:-1,:-1].T, y2.reshape((2*N1+2,N2+2))[:-1,:-1].T, a2[:-1,:-1].T, b2[:-1,:-1].T,color=np.sqrt(a2[:-1,:-1].T**2+b2[:-1,:-1].T**2),cmap ='viridis',arrowsize=0.8,density=1.5)
    colorbar(st3.lines)
    plt.plot([-H,H], [0,0], linewidth=2, color="k")
    S="fig_5_upper_plate"+str(t)+".png"
    plt.savefig(S, dpi=300)
    plt.close